**Классификация изображений с помощью сверточных нейронных сетей**

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6).
Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом *None*.
Необходимая точность (accuracy) сети на датасете CIFAR100 - 30%
Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [1]:
!pip install keras-flops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [3]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [5]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

169001437/169001437 [==============================] - 6s 0us/step


In [6]:
# Преобразуйте метки классов в one_hot формат
y_train = (y_train == np.arange(CLASSES)).astype(np.float32)
y_val = (y_val == np.arange(CLASSES)).astype(np.float32)

In [7]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [99]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(10, 3, padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=16, strides=12, padding = 'same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

In [100]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.6051e6


In [101]:
# вывод краткой информации о модели
model.summary()

Model: "sequential_41"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_42 (Conv2D)          (None, 32, 32, 10)        280       
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 3, 3, 10)         0         
 g2D)                                                            
                                                                 
 batch_normalization_41 (Bat  (None, 3, 3, 10)         40        
 chNormalization)                                                
                                                                 
 activation_82 (Activation)  (None, 3, 3, 10)          0         
                                                                 
 flatten_41 (Flatten)        (None, 90)                0         
                                                                 
 dense_41 (Dense)            (None, 100)             

In [102]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [103]:
# обучения модели
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=50
)

Epoch 1/50
391/391 [==============================] - 25s 62ms/step - loss: 3.7174 - accuracy: 0.1455 - val_loss: 3.4946 - val_accuracy: 0.1930
Epoch 2/50
391/391 [==============================] - 24s 61ms/step - loss: 3.2638 - accuracy: 0.2253 - val_loss: 3.3648 - val_accuracy: 0.2069
Epoch 3/50
391/391 [==============================] - 24s 61ms/step - loss: 3.1425 - accuracy: 0.2475 - val_loss: 3.1898 - val_accuracy: 0.2395
Epoch 4/50
391/391 [==============================] - 24s 61ms/step - loss: 3.0656 - accuracy: 0.2637 - val_loss: 3.1247 - val_accuracy: 0.2545
Epoch 5/50
391/391 [==============================] - 24s 61ms/step - loss: 3.0193 - accuracy: 0.2723 - val_loss: 3.1097 - val_accuracy: 0.2587
Epoch 6/50
391/391 [==============================] - 24s 62ms/step - loss: 2.9856 - accuracy: 0.2795 - val_loss: 3.0678 - val_accuracy: 0.2677
Epoch 7/50
391/391 [==============================] - 24s 61ms/step - loss: 2.9585 - accuracy: 0.2844 - val_loss: 3.0684 - val_accuracy: